In [1]:
from elasticsearch import Elasticsearch
es = Elasticsearch(hosts='140.137.41.81', port=9200)

In [2]:
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import metrics
import operator
import math

In [3]:
#建立ling-江采淳02text資料庫

from elasticsearch import Elasticsearch
import json

def create_index(es):
    body = dict()
    body['settings'] = get_setting()
    body['mappings'] = get_mappings()
    print(json.dumps(body)) #可以用json.dumps輸出來看格式有沒又包錯
    es.indices.create(index='ling-江采淳02text', body=body)

def get_setting():
    settings = {
        "index": {
            "number_of_shards": 3,
            "number_of_replicas": 2
        }
    }
    return settings

def get_mappings():
    mappings = {
        "properties": {
            #編號
            "Number": {
                "type": "integer"
            },
            #編號
            "name": {
                "type": "keyword"
            },
            #影片類別
            "video type": {
                "type": "keyword"
            },
            #平均
            "average": {
                "type": "float"
            },
            #一階差
            "first order difference": {
                "type": "float"
            },
            #二階差
            "second order difference": {
                "type": "float"
            },
            #最大心率
            "max heart": {
                "type": "integer"
            },
            #最小心率
            "min heart": {
                "type": "integer"
            },
            #心率變化範圍
            "max to min scope heart": {
                "type": "integer"
            },
            #資訊熵
            "entropy of information": {
                "type": "float"
            },
            #均方根
            "root mean square": {
                "type": "float"
            },
            #標準化均值
            "standardize average": {
                "type": "float"
            }
        }
    }
    return mappings

if __name__ == "__main__":
    es = Elasticsearch(hosts='140.137.41.81', port=9200)
    create_index(es)

{"settings": {"index": {"number_of_shards": 3, "number_of_replicas": 2}}, "mappings": {"properties": {"Number": {"type": "integer"}, "name": {"type": "keyword"}, "video type": {"type": "keyword"}, "average": {"type": "float"}, "first order difference": {"type": "float"}, "second order difference": {"type": "float"}, "max heart": {"type": "integer"}, "min heart": {"type": "integer"}, "max to min scope heart": {"type": "integer"}, "entropy of information": {"type": "float"}, "root mean square": {"type": "float"}, "standardize average": {"type": "float"}}}}


In [4]:
#將ling-江采淳數據01的2nd進行計算並且將結果傳送到ling-江采淳02text資料庫

from elasticsearch import Elasticsearch
import json

# 正常(feel=naturl)心率取3個為一組
SamplesStartA = 0
SamplesStopA = 30

#A視頻(feel=happy)心率取3個為一組
SamplesStartB = 0
SamplesStopB = 30

#C視頻(feel=sad)心率取3個為一組
SamplesStartC = 0
SamplesStopC = 30

name="江采淳"

arra=[0]*3
x=0
arrb=[0]*3
y=0
arrd=[0]*3
z=0

A=3
B=3
D=3

for n in range(0,25): #迴圈取[a1,a11,a21,...],[a2,a22,a33...],[a3,a23,a33...]...共取25組
    arra=[0]*3
    x=0
    arrb=[0]*3
    y=0
    arrd=[0]*3
    z=0
    A=3
    B=3
    D=3
    SamplesStartA+=1
    SamplesStopA+=1
    SamplesStartB+=1
    SamplesStopB+=1
    SamplesStartC+=1
    SamplesStopC+=1
    for m in range(SamplesStartA,SamplesStopA,10):  #迴圈取a1,a11,a21,a31,a41...一組取3個
        def get_queryA():
            queryA = {
            "query": {
                "bool": {
                    "must": [{
                        "term": {
                            "name": "江采淳"
                        }
                    },
                        {
                        "term": {
                            "feel": "natural"
                        }
                    },
                    {
                        "term": {
                            "time": m
                        }
                    },
                        {
                        "term": {
                            "trial": "2nd"
                        }
                    }]
                }
            }
        }
            return queryA
        if __name__ == "__main__":
            es = Elasticsearch(hosts='140.137.41.81', port=9200)
            queryA = get_queryA()
            resultA = es.search(index='ling-江采淳數據01', body=queryA)
            resultaa=json.dumps(resultA, ensure_ascii=False)
            resultPulseA=json.loads(resultaa)
            print("A")
            print(resultPulseA['hits']['hits'][0]['_source']['pulse'])
            aa=resultPulseA['hits']['hits'][0]['_source']['pulse']
            arra[x]=aa
            if x==2:
                print(arra)
                x-=2
            else:
                x+=1
    for p in range(SamplesStartB,SamplesStopB,10):  #迴圈取A視頻心率a1,a11,a21,a31,a41...一組取3個
        def get_queryB():
            queryB = {
            "query": {
                "bool": {
                    "must": [{
                        "term": {
                            "name": "江采淳"
                        }
                    },
                        {
                        "term": {
                            "feel": "happy"
                        }
                    },
                    {
                        "term": {
                            "time": p
                        }
                    },
                        {
                        "term": {
                            "trial": "2nd"
                        }
                    }]
                }
            }
        }
            return queryB
        if __name__ == "__main__":
            es = Elasticsearch(hosts='140.137.41.81', port=9200)
            queryB = get_queryB()
            resultB = es.search(index='ling-江采淳數據01', body=queryB)
            resultbb=json.dumps(resultB, ensure_ascii=False)
            resultPulseB=json.loads(resultbb)
            print("B")
            print(resultPulseB['hits']['hits'][0]['_source']['pulse'])
            bb=resultPulseB['hits']['hits'][0]['_source']['pulse']
            arrb[y]=bb
            if y==2:
                print(arrb)
                y-=2
            else:
                y+=1
    for q in range(SamplesStartC,SamplesStopC,10):     #迴圈取C視頻心率a1,a11,a21,a31,a41...一組取3個
        def get_queryC():
            queryC = {
            "query": {
                "bool": {
                    "must": [{
                        "term": {
                            "name": "江采淳"
                        }
                    },
                        {
                        "term": {
                            "feel": "sad"
                        }
                    },
                    {
                        "term": {
                            "time": q
                        }
                    },
                        {
                        "term": {
                            "trial": "2nd"
                        }
                    }]
                }
            }
        }
            return queryC
        if __name__ == "__main__":
            es = Elasticsearch(hosts='140.137.41.81', port=9200)
            queryC = get_queryC()
            resultC = es.search(index='ling-江采淳數據01', body=queryC)
            resultcc=json.dumps(resultC, ensure_ascii=False)
            resultPulseC=json.loads(resultcc)
            print("C")
            print(resultPulseC['hits']['hits'][0]['_source']['pulse'])
            cc=resultPulseC['hits']['hits'][0]['_source']['pulse']
            arrd[z]=cc
            if z==2:
                print(arrd)
                z-=2
            else:
                z+=1
    #正常心率平均
    ave = 0.0
    for i in range(0,A):
        ave+=arra[i]
    ave = ave/A

    print("A視頻心率特徵 :")

    #平均
    aveB = 0.0
    for i in range(0,B):
        aveB +=arrb[i]
    aveB = aveB/B
    print("平均: ",aveB)

    #DiffB1 一階差的平均值
    diffB1 = 0.0
    meanB1 = 0.0
    #X[n-1]-X[n]的總和
    for i in range(0,(B-1)):
        meanB1+=abs(arrb[i+1]-arrb[i])
    diffB1 = meanB1/(B-1)
    print("一階差: ",diffB1)

    #DiffB2 二階差的平均值
    diffB2 = 0.0
    meanB2 = 0.0
    #X[n+2]-X[n]的總和
    for i in range(0,(B-2)):
        meanB2 += abs(arrb[i+2]-arrb[i])
    diffB2 = meanB2/(B-2)
    print("二階差: ",diffB2)

    #range 心率變化範圍
    scopeB = 0
    scopemaxB = 0
    scopeminB = 110
    for i in range(0,B):
        #求最大心率
        if arrb[i]>scopemaxB:
            scopemaxB = arrb[i]
        if arrb[i]<scopeminB:
            scopeminB = arrb[i]
    scopeB = scopemaxB-scopeminB
    print("最大心率: ",scopemaxB)
    print("最小心率: ",scopeminB)
    print("心率變化範圍: ",scopeB)

    #H(X)資訊熵
    x = 0
    HB = 0
    arrpb = [0.0]*B
    for i in range(0,B):
        x=0 #初值為0
        #心率出現次數
        for j in range(0,B):
            if arrb[i]==arrb[j]:
                x+=1
        #p[i]機率
        arrpb[i] = x*1.0/B
    for k in range(0,B):
        HB += -arrpb[k]*math.log(arrpb[k],2)
    print("資訊熵: ",HB)

    #Radrm 均方根
    radrmB = 0.0   
    meanrB = 0.0
    #相鄰值平方差的總和
    for i in range(0,(B-1)):
        radrmB += pow(arrb[i+1]-arrb[i],2)
    meanrB = radrmB/B
    print("均方根: ",meanrB)

    #標準化均值
    arrbb=[0.0]*B
    aveb = 0.0
    for i in range(0,B):
        arrbb[i] = arrb[i]-ave
        aveb += arrbb[i]
    aveb = aveb/B
    print("標準化均值: ",aveb)
    def load_datas():
        datas = list()
        datas.append(
        {
            #匯入編號
            "Number": int(n),
            #匯入名字
            "name": name,
            #匯入影片類別
            "video type": "A",
            #匯入平均
            "average": float(aveB),
            #匯入一階差
            "first order difference":float(diffB1),
            #匯入二階差
            "second order difference":float(diffB2),
            #匯入最大心率
            "max heart":int(scopemaxB),
            #匯入最小心率
            "min heart":int(scopeminB),
            #匯入心率變化範圍
            "max to min scope heart":int(scopeB),
            #匯入資訊熵
            "entropy of information":float(HB),
            #匯入均方根
            "root mean square":float(meanrB),
            #匯入標準化均值
            "standardize average":float(aveb),
        }
        )
        return datas

    def create_data(es, datas):
        for data in datas:
            es.index(index='ling-江采淳02text', body=data) 

    if __name__ == "__main__":
        es = Elasticsearch(hosts='140.137.41.81', port=9200)
        datas = load_datas()
        create_data(es, datas)
    print("C視頻心率特徵: ")

    #平均
    aveD = 0.0
    for i in range(0,D):
        aveD +=arrd[i]
    aveD = aveD/D
    print("平均: ",aveD)

    #DiffD1 一階差的平均值
    diffD1 = 0.0
    meanD1 = 0.0
    #X[n+1]-X[n]的總和
    for i in range(0,(D-1)):
        meanD1 += abs(arrd[i+1]-arrd[i])
    diffD1 = meanD1/(D-1)
    print("一階差: ",diffD1)

    #DiffD2 二階差的平均值
    diffD2 = 0.0
    meanD2 = 0.0
    #X[n+2]-X[n]的總和
    for i in range(0,(D-2)):
        meanD2 += abs(arrd[i+2]-arrd[i])
    diffD2 = meanD2/(D-2)
    print("二階差: ",diffD2)

    #range 心率變化範圍
    scopeD = 0
    scopemaxD = 0
    scopeminD = 110
    for i in range(0,D):
        #求最大心率
        if arrd[i]>scopemaxD:
            scopemaxD = arrd[i]
        #求最小心率
        if arrd[i]<scopeminD:
            scopeminD = arrd[i]
    scopeD = scopemaxD-scopeminD
    print("最大心率: ",scopemaxD)
    print("最小心率: ",scopeminD)
    print("心率變化範圍: ",scopeD)

    #H(X)資訊熵
    x = 0
    HD = 0
    arrpd = [0.0]*D
    for i in range(0,D):
        x = 0 #初值為0 
        #心率出現次數
        for j in range(0,D):
            if arrd[i]==arrd[j]:
                x+=1
        arrpd[i] = x*1.0/D
    for k in range(0,D):
        HD += -arrpd[k]*math.log(arrpd[k],2)
    print("資訊熵: ",HD)

    #Radrm 均方根
    radrmD = 0.0
    meanrD = 0.0
    #相鄰值平方差的總和
    for i in range(0,(D-1)):
        radrmD += pow(arrd[i+1]-arrd[i],2)
    meanrD = radrmD/D
    print("均方根: ",meanrD)

    #標準化均值
    arrdd = [0.0]*D
    aved = 0.0
    for i in range(0,D):
        arrdd[i] = arrd[i] -ave
        aved +=arrdd[i]
    aved = aved/D
    print("標準化均值: ",aved)
    
    #匯入資料
    def load_datas():
        datas = list()
        datas.append(
        {
            #匯入編號
            "Number": int(n),
            #匯入名字
            "name": name,
            #匯入影片類別
            "video type": "C",
            #匯入平均
            "average": float(aveD),
            #匯入一階差
            "first order difference":float(diffD1),
            #匯入二階差
            "second order difference":float(diffD2),
            #匯入最大心率
            "max heart":int(scopemaxD),
            #匯入最小心率
            "min heart":int(scopeminD),
            #匯入心率變化範圍
            "max to min scope heart":int(scopeD),
            #匯入資訊熵
            "entropy of information":float(HD),
            #匯入均方根
            "root mean square":float(meanrD),
            #匯入標準化均值
            "standardize average":float(aved),
        }
        )
        return datas

    def create_data(es, datas):
        for data in datas:
            es.index(index='ling-江采淳02text', body=data)

    if __name__ == "__main__":
        es = Elasticsearch(hosts='140.137.41.81', port=9200)
        datas = load_datas()
        create_data(es, datas)

A
89
A
85
A
79
[89, 85, 79]
B
65
B
63
B
79
[65, 63, 79]
C
80
C
77
C
79
[80, 77, 79]
A視頻心率特徵 :
平均:  69.0
一階差:  9.0
二階差:  14.0
最大心率:  79
最小心率:  63
心率變化範圍:  16
資訊熵:  1.584962500721156
均方根:  86.66666666666667
標準化均值:  -15.333333333333329
C視頻心率特徵: 
平均:  78.66666666666667
一階差:  2.5
二階差:  1.0
最大心率:  80
最小心率:  77
心率變化範圍:  3
資訊熵:  1.584962500721156
均方根:  4.333333333333333
標準化均值:  -5.666666666666662
A
88
A
83
A
78
[88, 83, 78]
B
60
B
62
B
78
[60, 62, 78]
C
78
C
76
C
78
[78, 76, 78]
A視頻心率特徵 :
平均:  66.66666666666667
一階差:  9.0
二階差:  18.0
最大心率:  78
最小心率:  60
心率變化範圍:  18
資訊熵:  1.584962500721156
均方根:  86.66666666666667
標準化均值:  -16.333333333333332
C視頻心率特徵: 
平均:  77.33333333333333
一階差:  2.0
二階差:  0.0
最大心率:  78
最小心率:  76
心率變化範圍:  2
資訊熵:  1.3082708345352603
均方根:  2.6666666666666665
標準化均值:  -5.666666666666667
A
90
A
82
A
80
[90, 82, 80]
B
58
B
82
B
79
[58, 82, 79]
C
77
C
75
C
79
[77, 75, 79]
A視頻心率特徵 :
平均:  73.0
一階差:  13.5
二階差:  21.0
最大心率:  82
最小心率:  58
心率變化範圍:  24
資訊熵:  1.584962500721156
均方根:  195.0
標準化均值: 

C視頻心率特徵: 
平均:  79.0
一階差:  1.0
二階差:  2.0
最大心率:  80
最小心率:  78
心率變化範圍:  2
資訊熵:  1.584962500721156
均方根:  0.6666666666666666
標準化均值:  -2.6666666666666714
A
80
A
81
A
83
[80, 81, 83]
B
79
B
75
B
80
[79, 75, 80]
C
79
C
80
C
83
[79, 80, 83]
A視頻心率特徵 :
平均:  78.0
一階差:  4.5
二階差:  1.0
最大心率:  80
最小心率:  75
心率變化範圍:  5
資訊熵:  1.584962500721156
均方根:  13.666666666666666
標準化均值:  -3.3333333333333286
C視頻心率特徵: 
平均:  80.66666666666667
一階差:  2.0
二階差:  4.0
最大心率:  83
最小心率:  79
心率變化範圍:  4
資訊熵:  1.584962500721156
均方根:  3.3333333333333335
標準化均值:  -0.666666666666662
A
81
A
80
A
82
[81, 80, 82]
B
80
B
77
B
82
[80, 77, 82]
C
74
C
81
C
79
[74, 81, 79]
A視頻心率特徵 :
平均:  79.66666666666667
一階差:  4.0
二階差:  2.0
最大心率:  82
最小心率:  77
心率變化範圍:  5
資訊熵:  1.584962500721156
均方根:  11.333333333333334
標準化均值:  -1.3333333333333333
C視頻心率特徵: 
平均:  78.0
一階差:  4.5
二階差:  5.0
最大心率:  81
最小心率:  74
心率變化範圍:  7
資訊熵:  1.584962500721156
均方根:  17.666666666666668
標準化均值:  -3.0
A
84
A
81
A
83
[84, 81, 83]
B
80
B
76
B
83
[80, 76, 83]
C
75
C
80
C
78
[75, 80, 78